In [11]:
#GDD calculation for Soy from class SoyGDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class SoyGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def soy_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


Min/Max temp to calculate GDD from GEMS exchange

In [12]:
import pandas as pd
Becker_GEMS = pd.read_csv("becker_gems_2023_2024.csv", on_bad_lines='skip')

In [13]:
Becker_GEMS_2023 = SoyGDD(Becker_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [14]:
#Using year 2023 start date and end date for mine gdd
start_date = pd.to_datetime("5-10-2023").date()
end_date = pd.to_datetime("10-9-2023").date()

Soy AGDD for Becker 2023

In [16]:
Soy_gdd_BeckerGEMS2023 = Becker_GEMS_2023.soy_gdd(start_date, end_date)
print(Soy_gdd_BeckerGEMS2023)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
129      15 2023-05-10   11.8   55  441  329      120      919      927   
130      51 2023-05-11   13.2   55  443  330      120      919      929   
131      93 2023-05-12   15.3   55  444  332      120      919      931   
132     100 2023-05-13   11.2   55  445  333      120      919      933   
133      33 2023-05-14    5.3   55  447  334      120      919      935   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
277      50 2023-10-05    6.3   38  318  182      105      852      643   
278      71 2023-10-06    3.0   37  316  180      105      851      637   
279      85 2023-10-07    2.2   37  314  178      105      849      631   
280      72 2023-10-08    2.1   37  312  175      104      847      625   
281       0 2023-10-09    0.0   37  310  173      104      845      619   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
129     77.54  ...   